In [2]:
#Super-TicTacToeGame
#AI-Assignment#2
#BAI-4A

#Players : 1. User
#         2. AI (opponent)

#Group Members : 1. MUDASIR (22k-8732)
#                2. MAISUM ABBASS (22k-4129)
#               3. IRTEZA (22k-8731)

import tkinter as tk
from tkinter import font
from math import inf
from collections import Counter
import itertools
from time import time

class SuperTicTacToeGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("Super Tic-Tac-Toe Game")

        # Initialize game state
        self.state = "." * 81
        self.depth = 5
        self.box_won = ["."] * 9
        self.possible_goals = [(0, 4, 8), (2, 4, 6)]
        self.possible_goals += [(i, i+3, i+6) for i in range(3)]
        self.possible_goals += [(3*i, 3*i+1, 3*i+2) for i in range(3)]
        self.bot_move = -1

        # Create GUI elements
        self.create_board()
        self.create_display()

    def create_board(self):
        self.board_buttons = []
        for i in range(9):
            row = []
            for j in range(9):
                frame = tk.Frame(self.master, bg="white", bd=2)
                frame.grid(row=i, column=j, padx=1, pady=1)
                button = tk.Button(frame, text="", width=4, height=2, font=("Helvetica", 12, "bold"), command=lambda row=i, col=j: self.handle_click(row, col))
                button.pack(expand=True, fill=tk.BOTH)
                row.append(button)
            self.board_buttons.append(row)

    def create_display(self):
        self.display = tk.Label(self.master, text="Super-TicTacToe", font=("Times", 32, "bold"))
        self.display.grid(row=9, columnspan=9)

    def handle_click(self, row, col):
        if self.state[self.index(row, col)] == ".":
            # Player move
            self.state = self.add_piece(self.state, (row, col), "X")
            self.update_board()

            # Check if mini board is won
            if self.check_small_box(self.box_won) == ".":
                # Bot move
                self.bot_state, self.bot_move = self.minimax(self.state, (row, col), "O", self.depth, time())
                self.state = self.bot_state
                self.update_board()

    def index(self, x, y):
        return x * 9 + y

    def add_piece(self, state, move, player):
        x, y = move
        index = self.index(x, y)
        return state[:index] + player + state[index + 1:]

    def update_board(self):
        self.box_won = self.update_box_won(self.state)
        for i in range(9):
            for j in range(9):
                cell_state = self.state[self.index(i, j)]
                if cell_state == "X":
                    self.board_buttons[i][j].config(text="X", state=tk.DISABLED)
                elif cell_state == "O":
                    self.board_buttons[i][j].config(text="O", state=tk.DISABLED)
                else:
                    self.board_buttons[i][j].config(text="")

    def update_box_won(self, state):
        temp_box_win = ["."] * 9
        for b in range(9):
            idxs_box = self.indices_of_box(b)
            box_str = "".join(state[idx] for idx in idxs_box)
            temp_box_win[b] = self.check_small_box(box_str)
        return temp_box_win

    def indices_of_box(self, b):
        return list(range(b*9, b*9 + 9))

    def check_small_box(self, box_str):
        for idxs in self.possible_goals:
            (x, y, z) = idxs
            if (box_str[x] == box_str[y] == box_str[z]) and box_str[x] != ".":
                return box_str[x]
        return "."

    def possible_moves(self, last_move):
        if not isinstance(last_move, int):
            last_move = self.index(*last_move)
        box_to_play = last_move % 9
        idxs = self.indices_of_box(box_to_play)
        if self.box_won[box_to_play] != ".":
            pi_2d = [self.indices_of_box(b) for b in range(9) if self.box_won[b] == "."]
            possible_indices = list(itertools.chain.from_iterable(pi_2d))
        else:
            possible_indices = idxs
        return possible_indices

    def successors(self, state, player, last_move):
        succ = []
        moves_idx = []
        possible_indexes = self.possible_moves(last_move)
        for idx in possible_indexes:
            if state[idx] == ".":
                moves_idx.append(idx)
                succ.append(self.add_piece(state, (idx // 9, idx % 9), player))
        return zip(succ, moves_idx)

    def opponent(self, p):
        return "O" if p == "X" else "X"

    def evaluate_small_box(self, box_str, player):
        score = 0
        three = Counter(player * 3)
        two = Counter(player * 2 + ".")
        one = Counter(player * 1 + "." * 2)
        three_opponent = Counter(self.opponent(player) * 3)
        two_opponent = Counter(self.opponent(player) * 2 + ".")
        one_opponent = Counter(self.opponent(player) * 1 + "." * 2)

        for idxs in self.possible_goals:
            (x, y, z) = idxs
            current = Counter([box_str[x], box_str[y], box_str[z]])
            if current == three:
                score += 100
            elif current == two:
                score += 10
            elif current == one:
                score += 1
            elif current == three_opponent:
                score -= 100
                return score
            elif current == two_opponent:
                score -= 10
            elif current == one_opponent:
                score -= 1
        return score

    def evaluate(self, state, last_move, player):
        score = 0
        score += self.evaluate_small_box("".join(state[i] for i in range(len(state))), player) * 200
        for b in range(9):
            idxs = self.indices_of_box(b)
            box_str = "".join(state[idx] for idx in idxs)
            score += self.evaluate_small_box(box_str, player)
        return score

    def minimax(self, state, last_move, player, depth, s_time):
        succ = self.successors(state, player, last_move)
        best_move = (-inf, None)
        for s in succ:
            val = self.min_turn(s[0], s[1], self.opponent(player), depth - 1, s_time, -inf, inf)
            if val > best_move[0]:
                best_move = (val, s)
        return best_move[1]

    def min_turn(self, state, last_move, player, depth, s_time, alpha, beta):
        if depth <= 0 or self.check_small_box(self.box_won) != ".":
            return self.evaluate(state, last_move, self.opponent(player))
        succ = self.successors(state, player, last_move)
        for s in succ:
            val = self.max_turn(s[0], s[1], self.opponent(player), depth - 1, s_time, alpha, beta)
            if val < beta:
                beta = val
            if alpha >= beta:
                break
        return beta

    def max_turn(self, state, last_move, player, depth, s_time, alpha, beta):
        if depth <= 0 or self.check_small_box(self.box_won) != ".":
            return self.evaluate(state, last_move, player)
        succ = self.successors(state, player, last_move)
        for s in succ:
            val = self.min_turn(s[0], s[1], self.opponent(player), depth - 1, s_time, alpha, beta)
            if alpha < val:
                alpha = val
            if alpha >= beta:
                break
        return alpha

root = tk.Tk()
game = SuperTicTacToeGUI(root)
root.mainloop()
